In [14]:
using DataFrames

# To run this notebook, you need to have a data file available.
# You can either run the phenotype preprocessing scripts in ../preprocessing directory
# Or scp -r username@sherlock.stanford.edu:/scratch/PI/dpwall/DATA/phenotypes/jsonschema ../data

# Data
df = readtable("../data/all_samples_filtered_train.csv", header=false)
m, n = size(df)

(16291, 250)

In [15]:
# Form sparse array
all_data = sparse(Array(df))
#all_data[all_data.>1] = 1
dropzeros!(all_data)
p = size(nonzeros(all_data), 1)

1873895

In [19]:
using LowRankModels
losses = Array{Loss}(n)
for i=1:n
    options = unique(all_data[:, i])
    if length(options) == 3
        losses[i] = HingeLoss()
    else
        losses[i] = OrdinalHingeLoss(-1, maximum(options))
    end
end

In [21]:
using LowRankModels

k=4

# k-means
#rx = UnitOneSparseConstraint() # k-means
#ry = ZeroReg()

# soft k-means
#losses = HingeLoss()
rx = SimplexConstraint() 
ry = ZeroReg()

glrm = GLRM(all_data, losses, rx, ry, k);
#glrm.X = X
#glrm.Y = Y
X,Y,ch = fit!(glrm, SparseProxGradParams(max_iter=500), verbose=true); # fit GLRM

LowRankModels.SparseProxGradParams(1.0, 500, 1, 1.0e-5, 0.01)
Fitting GLRM
obj went up to 2.328053326069455e6; reducing step size to 0.8933970937500003
obj went up to 2.256557994053799e6; reducing step size to 0.5955980625000002
Iteration 10: objective value = 2.1977730767660253e6
obj went up to 2.1979275169563936e6; reducing step size to 0.4377645759375002
obj went up to 2.1548872879914553e6; reducing step size to 0.3378448114797658
obj went up to 2.123959399867158e6; reducing step size to 0.27376831992248474
Iteration 20: objective value = 2.1228065734568913e6
obj went up to 2.0958108855044567e6; reducing step size to 0.24458382137956813
Iteration 30: objective value = 2.0745734272827941e6
obj went up to 2.0734374609662439e6; reducing step size to 0.21851047519877534
Iteration 40: objective value = 2.052077297193754e6
obj went up to 2.0527386161238889e6; reducing step size to 0.19521662349651736
obj went up to 2.0339343273630661e6; reducing step size to 0.1744059640834766
Iteration 5

Stacktrace:
 [1] depwarn(::String, ::Symbol) at ./deprecated.jl:70
 [2] Array(::Type{Union{Int64, Range{Int64}}}, ::Int64) at ./deprecated.jl:57
 [3] get_yidxs(::Array{LowRankModels.Loss,1}) at /Users/kelley/.julia/v0.6/LowRankModels/src/losses.jl:77
 [4] (::LowRankModels.#kw##objective)(::Array{Any,1}, ::LowRankModels.#objective, ::LowRankModels.GLRM, ::Array{Float64,2}, ::Array{Float64,2}) at ./<missing>:0
 [5] #objective#61 at /Users/kelley/.julia/v0.6/LowRankModels/src/evaluate_fit.jl:82 [inlined]
 [6] (::LowRankModels.#kw##objective)(::Array{Any,1}, ::LowRankModels.#objective, ::LowRankModels.GLRM) at ./<missing>:0
 [7] #fit!#85(::LowRankModels.ConvergenceHistory, ::Bool, ::Array{Any,1}, ::Function, ::LowRankModels.GLRM, ::LowRankModels.SparseProxGradParams) at /Users/kelley/.julia/v0.6/LowRankModels/src/algorithms/sparse_proxgrad.jl:50
 [8] (::StatsBase.#kw##fit!)(::Array{Any,1}, ::StatsBase.#fit!, ::LowRankModels.GLRM, ::LowRankModels.SparseProxGradParams) at ./<missing>:0
 [9] 

In [22]:
# Output to file
X_df = convert(DataFrame, X.')
writetable("../data/softkmeans_ordhinge_X$(k)_12.30.17.csv", X_df, separator = ',', header=false)
Y_df = convert(DataFrame, Y)
print(size(names(df)), size(Y_df))
writetable("../data/softkmeans_ordhinge_Y$(k)_12.30.17.csv", Y_df, separator = ',', header=false)

(250,)(6, 250)